In [ ]:
%pip install pdfplumber
import requests
import pdfplumber
import tempfile
import re
from pyspark.sql.functions import lit,trim

In [2]:
blob_relative_path = 'www.sba.gov/sites/sbagov/files/2023-01'
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from pyspark.sql.functions import split,col,substring,expr,first
import concurrent.futures
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
# Azure storage access info 

blob_account_name = 'usafactsbronze' # replace with your blob name 
blob_container_name = 'bronze' # replace with your container name 
linked_service_name = 'bronze' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
bronze_wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

blob_service_client = BlobServiceClient("https://{}.blob.core.windows.net".format(blob_account_name), credential=blob_sas_token)


In [3]:
def get_file_paths(dir_path):
    file_paths = []

    def recursive_search(directory):
        files = mssparkutils.fs.ls(directory)
        for file in files:
            if file.isDir:
                recursive_search(file.path)
            else:
                if file.path.endswith(('.pdf')):
                    path = file.path.split(dir_path.split(blob_relative_path)[0])[-1]
                    file_paths.append(path)
    
    recursive_search(dir_path)
    return file_paths


In [19]:
blob_relative_path = 'www.sba.gov/sites/sbagov/files/2023-01'
blob_account_name = 'usafactssilver' # replace with your blob name 
blob_container_name = 'silver' # replace with your container name 
linked_service_name = 'silver' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
silver_wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 


In [20]:
print(silver_wasbs_path)

In [22]:
links = get_file_paths(bronze_wasbs_path)
print(len(links))
print(links)

In [5]:
def exclude_and_return(data):
    remaining_lists = []
    exclude_mode = False

    for sublist in data:
        if not exclude_mode and isinstance(sublist, list) and sublist[0] == 'This':
            exclude_mode = True
        elif exclude_mode and isinstance(sublist, list) and sublist[-1] == 'programs.':
            exclude_mode = False
        elif not exclude_mode:
            remaining_lists.append(sublist)

    return remaining_lists if not exclude_mode else None


In [6]:
def process_list_of_lists(list_of_lists,length):
    result = []
    l = length - 1
    for inner_list in list_of_lists:
        if len(inner_list) == length:
            result.append(inner_list)
        elif len(inner_list) > length:
            #concat_list = inner_list[:-1] + inner_list[-2:10:-1]
            remaining = inner_list[-l:]
            #print(remaining)
            concat_list = ['_'.join(inner_list[:-l])] + remaining
            result.append(concat_list)
        elif 1 <= len(inner_list) <= 3:
            concat_list = ['_'.join(inner_list)]
            result.append(concat_list)
    
    return result

In [7]:
def process_first_column(out):
    if isinstance(out[0], list) and len(out[1]) == 1:
        index_gb = -1
        for i, sublist in enumerate(out):
            if 'Guarantied_Business' in sublist:
                index_gb = i
                break
        if index_gb != -1:
            gb = out[index_gb][0]  # Extract 'Guarantied_Business'
            for i in range(index_gb + 1, len(out)):
                if len(out[i]) == 1:
                    out[i][0] = '_'.join([gb] + out[i])
        output = []
        current_key = ''
        for item in out[1:]:
            if len(item) == 1:
                current_key = item[0]
            else:
                output.append([f'{current_key}_{item[0]}'] + item[1:])
    else:
        output = []
        current_key = ''

        for item in out:
            if len(item) == 1:
                current_key = item[0]
            else:
                output.append([f'{current_key}_{item[0]}'] + item[1:])
        # for i in output:
        #     print(i)
    return output


In [8]:
def process_2_length_1(data):
    new_list = []
    i = 0
    while i in range(len(data) - 1):
    #while i < len(data) - 1:
        sublist = data[i]
        next_sublist = data[i + 1] if i + 1 < len(data) else None
        if len(sublist) == 1 and next_sublist and len(next_sublist) > 1:
            temp = sublist[0]  # Grabbing the only element from the current sublist
            #new_list.append([f'{temp}_{next_sublist[0]}'] + next_sublist[1:])        
            #new_list.append(l)
            i+=1
            while i < len(data) and len(data[i]) > 1:
                #temp = merged
                #temp = merged[0].split('_')[1]
                merged = [f'{temp}_{data[i][0]}'] + data[i][1:]
                i += 1        
                new_list.append(merged)
        else:
            new_list.append(sublist)
            i += 1       

    output = []
    current_key = ''

    for item in new_list:
        if len(item) == 1:
            current_key = item[0]
        else:
            output.append([f'{current_key}_{item[0]}'] + item[1:])
    # for i in output:
    #     print(i)
    return output
    #print(new_list)    
    # for l in new_list:
    #     print(l)   

In [31]:

links = ['www.sba.gov/sites/sbagov/files/2023-01/WDS_PostChargeOffRecoveryRates_Report_20221231-508.pdf']

In [36]:
import pandas as pd
from pyspark.sql.functions import when,col,lit
bad_records = []
for i in links:
    try:
        url = f'https://usafactsbronze.blob.core.windows.net/bronze/{i}'
        print(url)
        response = requests.get(url)
        pdf_file = response.content
        #print(pdf_file)
        with tempfile.NamedTemporaryFile(suffix='.pdf') as tmp_file:
            tmp_file.write(pdf_file)
            tmp_file.seek(0)
            with pdfplumber.open(tmp_file.name) as pdf:
                pages = pdf.pages
                df2 = None
                full_data = []
                for page in pages :
                    tables = page.extract_text()
                    full_data.append(tables)

        #print(full_data)

        result_string = [', '.join(full_data).replace(' ', ';').replace('\n', ';')]
        result_lists = [tuple(s.split(';')) for s in result_string]
        #print(result_lists)

        original_list = [element.strip(',') for element in result_lists[0]]
        #print(original_list)

        end_index = original_list.index('programs.')
        start_index = original_list.index('This')  # Adding 1 to make sure end_index is greater than start_index

        end_index1 = original_list.index('Year')
        start_index1 = original_list.index('Table')
        #extracted_data = [row[start_index:end_index] for row in original_list]
        Header_data = original_list[start_index1:end_index1 +1]
        footer_data = original_list[start_index:end_index +1]
        Header_data = ' '.join(Header_data)
        #print(Header_data)
        footer_data = ' '.join(footer_data)
        #print(footer_data)

        # Split the data into lines
        consolidatad_data = []
        for data in full_data:    
            lines = data.split('\n')
            #consolidatad_data.append(lines)
            #print(lines)
            
            for data in lines:
                #for i in data:
                consolidatad_data.append(data)
        #print(consolidatad_data)

        #for liness in lines:
        converted_data1 = [[c for c in item.split()] for item in consolidatad_data]
        converted_data = [sublist for sublist in converted_data1 if len(sublist) > 1 or any(len(str(item)) > 1 for item in sublist if isinstance(item, str))]
        #print(converted_data)
        column_header = (converted_data[2])

        ##### Column Header creation
        index = next((i for i, item in enumerate(column_header) if item.isdigit()), None)
        # Concatenate elements before the first numeric element
        concatenated = ['_'.join(column_header[:index])] + column_header[index:]
        #print(concatenated)
        #print(len(concatenated)-1)
        length = len(concatenated)
        #print(length)

        result = exclude_and_return(converted_data)
        filtered_list = [lst for lst in result if lst[0] not in ['Table', 'Fiscal', 'Purchase','Program','Charge']]

        datas = []
        for inner_list in filtered_list:
            if '(' in inner_list[0]:
                datas.append(inner_list)
            else:    
                inner_list[:] = [word for word in inner_list if '(' not in word]
                inner_list = [word.replace(')', '') for word in inner_list]
                datas.append(inner_list)
        #print(datas)
        out = process_list_of_lists(datas,length)
        #print(out)
        if len(out[0]) == 1 and len(out[1]) == 1:            
            x = process_2_length_1(out)
            #print(x)
        else:
            x = process_first_column(out)
        x =[sublist if not any(keyword in sublist[0] for keyword in ['Subtotal', 'Total']) else ['Subtotal' if 'Subtotal' in sublist[0] else 'Total'] + sublist[1:] for sublist in x]
        #print(x)
        data_frame = pd.DataFrame(x,columns=concatenated)
        #display(data_frame)
        spark_df = spark.createDataFrame(data_frame)
        spark_df = spark_df.withColumn("Header", lit(Header_data)).withColumn("Footer", lit(footer_data))
        silver_wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
        spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token)
        file_location = silver_wasbs_path + '/' + Header_data
        spark_df.write.format('delta').mode('overwrite').option("overwriteSchema", "true").option("path",file_location).save()
        print(file_location,'sucessfully uploaded')
        display(spark_df)
    except BaseException as e :
        bad_records.append((url,url.split('/')[-1],e))
        #bad_records.append(i)
        #bad_records.append((e))
        print(e)       

In [37]:
print(bad_records)